In [1]:
# config box
d = {"k1":"v1",
     "k2":"v2", 
     "k3":"v3", 
     "k4":"v4" }


d.keys()

dict_keys(['k1', 'k2', 'k3', 'k4'])

In [2]:
from box import ConfigBox

d1 = ConfigBox(d)
d1.k1

'v1'

In [3]:
type(d1)

box.config_box.ConfigBox

In [5]:
d1

ConfigBox({'k1': 'v1', 'k2': 'v2', 'k3': 'v3', 'k4': 'v4'})

In [6]:
# ensure_annotation

def get_predict(x:int, y:int)->int:
    return x*y

get_predict(3, "t")

'ttt'

In [ ]:
from ensure import ensure_annotations

@ensure_annotations
def get_predict_1(x:int, y:int)->int:
    return x*y

get_predict_1(3, "t")


In [9]:
import gdown

url = "https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing"

In [11]:
prefix = "https://drive.google.com/uc?/export=download&id="
file_code = url.split("/")[-2]
file_code

'1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3'

In [13]:
gdown.download(prefix+file_code, "dataset.zip")

Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=66e44a84-635e-425e-b15c-b5459b82b983
To: d:\AI\projects\DL_KIDNEY_DISEASE_CLASSIFICATION\research\dataset.zip
100%|██████████| 57.7M/57.7M [00:45<00:00, 1.27MB/s]


'dataset.zip'

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, get_size
from cnnClassifier import logger
import os
from zipfile import ZipFile


@dataclass(forzen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


        create_directories([self.config.artifact_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config





class DataIngestion:

    def __init__(self, config:DataIngestionConfig) -> None:
        self.config = config

    def download_data(self):
        try:
            URL = self.config.source_URL
            prefix = "https://drive.google.com/uc?/export=download&id="
            file_code = URL.split("/")[-2]
            gdown.download(prefix+file_code, self.config.local_data_file)
            logger.info(f"Dowmloaded data from {URL} into {self.config.local_data_file}")
        except Exception as e :
            raise e
        



    def unzip_file(self):
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with ZipFile.open(self.config.local_data_file, "r") as zip_ref:
                zip_ref.extractall(unzip_path)
        except Exception as e:
            raise e




In [ ]:
# pipeline intialization

try:
    data_ingestion_config = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH).get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_file()
except Exception as e:
    raise e